### Import libraries

In [ ]:
import flopy
import os
import flopy.utils.binaryfile as bf
import numpy as np
from osgeo import gdal
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tempfile import TemporaryDirectory
from pathlib import Path

### Create MODFLOW model simulation file

In [ ]:
name = "peat_testing_0"
workspace="../testing_peat_module/Model/"

sim = flopy.mf6.MFSimulation(sim_name = name, 
                            sim_ws=workspace)


### Set time discretization 

Set simulation for:
- 10 stress periods
- 5 days of stress period length
- 1 time steps in a stress period
- 1 multiplier length of stress period

In [ ]:
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(
    sim, nper=10,time_units="DAYS", perioddata=[[5,1,1] for _ in range(10)]
)

### Set iterative model solver (ims) and gwf package (?)

In [ ]:
ims = flopy.mf6.modflow.mfims.ModflowIms(sim, pname="ims", complexity="SIMPLE")
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)
model_nam_file = "{}.nam".format(name)

### Read raster input files for grid discretization

In [ ]:
# Raster paths
demPath = "../testing_peat_module/Raster//DEMNAS_1012-53_v1.0_48s_fill_100m_clip.tif"

# Open files
demDs = gdal.Open(demPath)

# Get data as arrays
demData = demDs.GetRasterBand(1).ReadAsArray()

### Set grid

In [ ]:
# spatial discretization object

# Nlay = 2
Nrow = demDs.RasterXSize
Ncol = demDs.RasterYSize
cell_size = demDs.GetGeoTransform()[1]
ztop = demDs.ReadAsArray()
zbot = np.linspace(-10/1, -10, 1 ) # set constant thickness of 10 units

dis = flopy.mf6.ModflowGwfdis(gwf, nlay=1, nrow=Nrow, ncol=Ncol, delr=cell_size, delc=cell_size,
                              top=ztop, botm=zbot)

# square model with a specified head equal to h1 along the edge of the model in layer 1. 
# A well is located in the center of the upper-left quadrant in layer 10. 
# First, set the name of the model and the parameters of the model: 
# the number of layers Nlay, 
# the number of rows and columns N, 
# lengths of the sides of the model L, 
# aquifer thickness H, hydraulic conductivity k, 
# and the well pumping rate q.

# h1 = 100
# Nlay = 10
# N = 101
# L = 400.0
# H = 50.0
# k = 1.0
# q = -1000.0

# bot = np.linspace(-H / Nlay, -H, Nlay)
# delrow = delcol = L / (N - 1)
# dis = flopy.mf6.ModflowGwfdis(
#     gwf,
#     nlay=Nlay,
#     nrow=N,
#     ncol=N,
#     delr=delrow,
#     delc=delcol,
#     top=0.0,
#     botm=bot,
# )

zbot

### Create initial condition package

In [ ]:
# Initial head
start = 3

ic = flopy.mf6.modflow.mfgwfic.ModflowGwfic(gwf, pname= "ic", strt=start)

### Set hydraulic conductivity (property flow)

In [ ]:
# Set hydraulic conductivity value
hk = [1E-6]
# Create node property flow package
npf = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf(
    gwf, pname="npf", icelltype=1, k=hk, save_flows=True
)

### Add RIV package

In [ ]:
#Add the drain package (DRN) to the MODFLOW model
river_path = "../testing_peat_module/Raster/Stream_DEMNAS.tif"
river = gdal.Open(river_path)
riv_nrow = river.RasterYSize
riv_ncol = river.RasterXSize
riv_cellsize = river.GetGeoTransform()[1]

list = []
for i in range(riv_nrow):
    for j in range(riv_ncol):
        value = river.GetRasterBand(1).ReadAsArray()[i][j]
        if value == 1:
            list.append([0, i, j, ztop[i][j], 0.001])  # layer, row, column, elevation, conductance

drn_spd = {0:list}
drn = flopy.mf6.ModflowGwfdrn(gwf, stress_period_data=drn_spd)



In [ ]:
# Create the constant head package.
# List information is created a bit differently for
# MODFLOW 6 than for other MODFLOW versions.  The
# cellid (layer, row, column, for a regular grid)
# must be entered as a tuple as the first entry.
# Remember that these must be zero-based indices!
N = 101
h1 = h2 = start

chd_rec = []
chd_rec.append(((0, int(N / 4), int (N / 4)), h2))
for layer in range(0, Nlay):
    for row_col in range(0, N):
        chd_rec.append(((layer, row_col, 0), h1))
        chd_rec.append(((layer, row_col, N - 1), h1))
        if row_col != 0 and row_col != N - 1:
            chd_rec.append(((layer, 0, row_col), h1))
            chd_rec.append(((layer, N - 1, row_col), h1))
chd = flopy.mf6.modflow.mfgwfchd.ModflowGwfchd(
    gwf,
    pname="chd",
    maxbound=len(chd_rec),
    stress_period_data=chd_rec,
    save_flows=True,
)

In [ ]:
sim.write_simulation()
print(os.listdir(workspace))


In [ ]:
# Run the simulation
success, buff = sim.run_simulation()
print("\nSuccess is: ", success)